### Server Side

In [ ]:
import socket
import cv2
import pickle # this pickle library provide function to convert data into bytes stream


cap=cv2.VideoCapture(0)
soc= socket.socket(socket.AF_INET, socket.SOCK_STREAM )
soc.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
# AF_INET corresponds to IPv4( means AddressFormat for Internet network IPv4 ) 
# SOCK_STREAM corresponds to TCP
hostname= socket.gethostname()
print(hostname)
host=socket.gethostbyname(hostname)
print(host)
soc.bind((host, 10001)) #socket binding
print("socket is binded")
soc.listen() #listen is any client want to connect
print("host ip {}".format(host))

obj, address= soc.accept()
print("client ip is {}".format(address))

while True:
    ret, cam= cap.read()
    ret, send_encoded = cv2.imencode(".jpg", cam)
    print("encoded and ready to convert in bytes and send to client")
    vid_bytes=pickle.dumps(send_encoded )# python objects (list, dict, etc.) into byte streams (0s and 1s)
    obj.send(vid_bytes)
    recv_encoded = obj.recv(10000000)
    print("encoded data is received from client")
    if recv_encoded != 0:
        data_recv = pickle.loads(recv_encoded)
        recved =cv2.imdecode(data_recv, cv2.IMREAD_COLOR)# covert image to BGR color format
        print("data has been decoded which is send by client")
        cv2.putText(recved,"Server",(20,40),cv2.FONT_HERSHEY_SIMPLEX, 1 ,(255,255,255),2)
        cv2.imshow("this is server and data receive from client", recved)
        if cv2.waitKey(5) == 13:
            break
            
    else:
        print("can't receive from client")
cv2.destroyAllWindows()